# Offline Solver

In [1]:
import os
import pickle
import pandas as pd
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [2]:
os.chdir("..")
import config
from core import osrm

## Load Data (Payload) for a single instance

In [3]:
with open("./data/offline_payload.pkl", 'rb') as f:
    payload = pickle.load(f)

date_str = payload['date']

## Offline Functions

In [4]:
def create_data_model(df_requests, df_driver_runs, date, depot):
    """
    Given a pandas.DataFrame of requests and a pandas.DataFrame of driver runs and the date will
    create the data model for the VRP optimization.

    Args:
        df_requests: (pandas.DataFrame)
        df_driver_runs: (pandas.DataFrame)
        date: (datetime.date or str in isoformat)
        depot: {'lat': , 'lon': }
    Returns:
        dictionary
    Raises:
        TODO
    """
    # generate locations, time_windows, pickups_deliveries, starts, ends, vehicle_capacities
    locations = []  # {'lat': 34, 'lon':- -85.1}
    time_windows = []  # [window_start, window_end]
    pickups_deliveries = []  # [source_location, target_location]
    starts = []  # location_id
    ends = []  # location_id
    am_capacities = []  # 8
    wc_capacities = [] # 3
    am_demands = []
    wc_demands = []
    booking_ids = []
    location_types = []

    current_location = 0
    for k, v in df_driver_runs.iterrows():
        locations.append(depot['pt'])
        locations.append(depot['pt'])
        am_capacities.append(v['am_capacity'])
        wc_capacities.append(v['wc_capacity'])
        time_windows.append([int(v['start_time'])+600, int(v['end_time'])-600])
        time_windows.append([int(v['start_time'])+600, int(v['end_time'])-600])
        starts.append(current_location)
        ends.append(current_location+1)
        booking_ids.append(-1)
        booking_ids.append(-1)
        location_types.append("depot_start")
        location_types.append("depot_end")
        am_demands.append(0)
        am_demands.append(0)
        wc_demands.append(0)
        wc_demands.append(0)
        current_location += 2

    for k, v in df_requests.iterrows():
        locations.append(v['pickup_pt'])
        locations.append(v['dropoff_pt'])
        time_windows.append([int(v['pickup_time_window_start']), int(v['pickup_time_window_end'])])
        time_windows.append([int(v['dropoff_time_window_start']), int(v['dropoff_time_window_end'])])
        pickups_deliveries.append([current_location, current_location+1])
        am_demands.append(v['am'])
        am_demands.append(-v['am'])
        wc_demands.append(v['wc'])
        wc_demands.append(-v['wc'])
        booking_ids.append(v['booking_id'])
        booking_ids.append(v['booking_id'])
        location_types.append("pickup")
        location_types.append("dropoff")
        current_location += 2
    # These functions will need to be created once server is set up
    time_matrix = osrm.request_travel_time_matrix(locations)
    distance_matrix = osrm.request_distance_matrix(locations)

    data = {
        "locations": locations,
        "time_windows": time_windows,
        "pickups_deliveries": pickups_deliveries,
        "starts": starts,
        "ends": ends,
        "am_capacities": am_capacities,
        "wc_capacities": wc_capacities,
        "am_demands": am_demands,
        "wc_demands": wc_demands,
        "booking_ids": booking_ids,
        "location_types": location_types,
        "time_matrix": time_matrix,
        "df_requests": df_requests,
        "df_driver_runs": df_driver_runs,
        "num_vehicles": len(starts),
        "date": date,
        "distance_matrix": distance_matrix
    }
    return data

In [5]:
def get_sum_of_all_distances(time_matrix, num_vehicles):
    result = 0
    for i in range(num_vehicles*2, len(time_matrix)):
        result += sum(time_matrix[i])
    return result

In [6]:
def extract_vehicle_runs(data, manager, routing, solution):
    """

    Args:
        data:
        manager:
        routing:
        solution:

    Returns:
        vehicle_run (list)
    """
    print(f'VRP: extract_vehicle_runs - Start')
    print(f"Objective: {solution.ObjectiveValue()}")
    time_dimension = routing.GetDimensionOrDie('Time')
    distance_dimension = routing.GetDimensionOrDie('Distance')
    total_time = 0
    vehicle_runs = []
    plan_outputs = []
    for vehicle_id in range(data['num_vehicles']):
        vehicle_cost = 0
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        vehicle_run = []
        i = 0
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            plan_output += '{0} Time({1},{2}) -> '.format(
                manager.IndexToNode(index), solution.Min(time_var),
                solution.Max(time_var))
            vehicle_run.append({'loc': manager.IndexToNode(index),
                                'early_time': solution.Min(time_var),
                                'late_time': solution.Max(time_var),
                                'order': i})
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            cost = routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
            vehicle_cost += cost
            i += 1
        time_var = time_dimension.CumulVar(index)
        vehicle_run.append({'loc': manager.IndexToNode(index),
                            'early_time': solution.Min(time_var),
                            'late_time': solution.Max(time_var),
                            'order': i})
        vehicle_runs.append(vehicle_run)
        plan_output += '{0} Time({1},{2})\n'.format(manager.IndexToNode(index),
                                                    solution.Min(time_var),
                                                    solution.Max(time_var))
        plan_output += 'Time of the route: {} seconds\n'.format(
            solution.Min(time_var))
        plan_output += f"Total Cost = {vehicle_cost}"
        plan_outputs.append(plan_output)
        print(f"plan_output for vehicle_id={vehicle_id}: {plan_output}")
        total_time += solution.Min(time_var)
        print(f'VRP: extract_vehicle_runs - End')
        
    return vehicle_runs

In [7]:
def prepare_manifests(data, vehicle_runs):
    print(f'VRP: prepare_manifests - Start')
    dfs = []
    for i in range(len(vehicle_runs)):
        vehicle_run = vehicle_runs[i]
        df = pd.DataFrame.from_records(vehicle_run)
        df['run_id'] = i
        dfs.append(df)
    result = pd.concat(dfs, ignore_index=True)
    temp = pd.DataFrame({'booking_id': data['booking_ids'],'action': data['location_types']})
    result = result.merge(temp, how='left', left_on='loc', right_index=True, validate='one_to_one')
    result['early_arrival_dt'] = result['early_time']
    result['scheduled_arrival_dt'] = result['late_time']
    result['state'] = "planned"
    if type(data['date']) == str:
        result['date'] = data['date']
    else:
        result['date'] = data['date'].isoformat()
    result['early_arrival_dt'] = result['scheduled_arrival_dt']
    result = result[['run_id', 'booking_id', 'order', 'early_arrival_dt', 'state','action', 'date']]
    return result

In [8]:
def run_vrp(data):
    """
    Given a data model generated by optimizer.create_data_model, will run a VRP. Returns the timetable
    as a pandas.DataFrame from this VRP run.

    Args:
        data: (dictionary)
    Returns:
        (pandas.DataFrame)
    Raises:
        TODO
    """
    print(f'VRP: Running VRP to return a Timetable')
    if config.VRP_TIME_LIMIT is None:
        print(f'VRP: Time Limit is None')
        vrp_time_limit = len(data['pickups_deliveries'])
    else:
        vrp_time_limit = config.VRP_TIME_LIMIT
        print(f'VRP: Time Limit is {vrp_time_limit}')
    print(f'VRP: time limit will be {vrp_time_limit}')

    # Create the routing index manager
    print(f'VRP: Create Routing Index Manager (pywrapcp.RoutingIndexManager) - Start')
    manager = pywrapcp.RoutingIndexManager(len(data['time_matrix']),
                                           data['num_vehicles'],
                                           data['starts'],
                                           data['ends'])
    print("VRP: Created Routing Index Manager")
    # Create routing model
    print(f'VRP: Creating Routing Model (pywrapcp.RoutingModel) - Start')
    routing = pywrapcp.RoutingModel(manager)
    print("VRP: Created Routing Model")

    max_distance_per_vehicle = get_sum_of_all_distances(data['distance_matrix'], data['num_vehicles'])
    max_time_per_vehicle = get_sum_of_all_distances(data['time_matrix'], data['num_vehicles'])

    # DISTANCE dimension
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    distance_callback_index = routing.RegisterTransitCallback(distance_callback)
    dimension_name = 'Distance'
    print(f'VRP: Adding Distance Dimension (routing.AddDimension) - Start')
    routing.AddDimension(
        distance_callback_index,
        0,  # slack TODO
        max_distance_per_vehicle,  # max distance per vehicle
        True,  # start cumul to zero
        dimension_name)
    # distance_dimension = routing.GetDimensionOrDie(dimension_name)
    print("VRP: Added Distance Dimension")

    # TIME dimension
    def time_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['time_matrix'][from_node][to_node] + config.DWELL_TIME

    transit_callback_index = routing.RegisterTransitCallback(time_callback)
    dimension_name = 'Time'
    print(f'VRP: Adding Time Dimension (routing.AddDimension) - Start')
    routing.AddDimension(
        transit_callback_index,
        config.MAX_VEHICLE_SLACK,  # slack TODO
        config.DAY_END_TIME,  # max time per vehicle
        False,  # start cumul to zero
        dimension_name)
    time_dimension = routing.GetDimensionOrDie(dimension_name)

    print("VRP: Added Time Dimension")

    # set objective
    print(f'VRP: SetArcCostEvalatorOfAllVehicles')
    if config.OBJECTIVE_IS_DISTANCE:
        routing.SetArcCostEvaluatorOfAllVehicles(distance_callback_index)
    else:
        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    print("VRP: Defined Transit Callback")

    # pickup and deliveries.
    print(f'VRP: Adding Pickup/Delivery indexes for each request - Start')
    for request in data['pickups_deliveries']:
        pickup_index = manager.NodeToIndex(request[0])
        delivery_index = manager.NodeToIndex(request[1])
        routing.AddPickupAndDelivery(pickup_index, delivery_index)
        routing.solver().Add(
            routing.VehicleVar(pickup_index) == routing.VehicleVar(
                delivery_index))
        routing.solver().Add(
            time_dimension.CumulVar(pickup_index) <=
            time_dimension.CumulVar(delivery_index))

    print("VRP: Finished with Pickup & Deliveries")

    # time windows (trips)
    print("VRP: Adding Time Windows - Start")
    for time_window_index in range(0, len(data['time_windows'])):
        if (time_window_index in data['starts'] + data['ends']):
            continue
        index = manager.NodeToIndex(time_window_index)
        v = data['time_windows'][time_window_index]
        time_dimension.CumulVar(index).SetRange(v[0], v[1])

    print("VRP: Completed Time Windows")

    # time windows (start and end locations)
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        start_loc = data['starts'][vehicle_id]
        time_dimension.CumulVar(index).SetRange(
            data['time_windows'][start_loc][0],
            data['time_windows'][start_loc][1])
        index = routing.End(vehicle_id)
        end_loc = data['ends'][vehicle_id]
        time_dimension.CumulVar(index).SetRange(
            data['time_windows'][end_loc][0],
            data['time_windows'][end_loc][1])
    print("VRP: Completed Time Windows (Start and End Locations)")


    # Instantiate route start and end times to produce feasible times.
    print("VRP: Instantiate Route Start and End Times - Start")
    for i in range(data['num_vehicles']):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i)))
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.End(i)))

    print("VRP: Instantiated Route Start and End Times")

    # AM demand constraints
    def am_demand_callback(from_index):
        from_node = manager.IndexToNode(from_index)
        return data['am_demands'][from_node]
    print(f'VRP: am_demand_callback - Starting')
    am_demand_callback_index = routing.RegisterUnaryTransitCallback(
        am_demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        am_demand_callback_index,
        0,  # null capacity slack
        data['am_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'AMCapacity')

    print("VRP: Complete AM Capacity Constraints")

    # WC demand constraints
    def wc_demand_callback(from_index):
        from_node = manager.IndexToNode(from_index)
        return data['wc_demands'][from_node]

    print(f'VRP: Starting wc_demand_callback')
    wc_demand_callback_index = routing.RegisterUnaryTransitCallback(
        wc_demand_callback)
    print(f'VRP: Ended wc_demand_callback')
    routing.AddDimensionWithVehicleCapacity(
        wc_demand_callback_index,
        0,  # null capacity slack
        data['wc_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'WCCapacity')

    print("VRP: Completed Capacity Constraints")

    # Allow to drop nodes
    if config.OBJECTIVE_IS_DISTANCE:
        penalty = max_distance_per_vehicle
    else:
        penalty = max_time_per_vehicle
    print(f"VRP Penalty={penalty}")
    for node in range(data['num_vehicles'] * 2, len(data['time_matrix'])):
        routing.AddDisjunction([manager.NodeToIndex(node)], penalty)

    print("VRP: Completed Allowing to Drop Nodes")

    # search parameters
    print(f'VRP: Assigning Search Parameters (pywrapcp.DefaultRoutingSearchParameters)')
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        config.FIRST_SOLUTION_STRATEGY)
    search_parameters.local_search_metaheuristic = (
        config.LOCAL_SEARCH_METAHEURISTIC)
    search_parameters.time_limit.seconds = vrp_time_limit

    print("VRP: Completed Search Parameters")
    

    # Solve the problem.
    print("VRP: Get Solution (routing.SolveWithParameters) - Start")
    search_parameters.log_search = config.LOG_SEARCH
    solution = routing.SolveWithParameters(search_parameters)

    print("VRP: Extract Veh. Runs & Manifests from Solution")
    vehicle_runs = extract_vehicle_runs(data, manager, routing, solution)
    manifests = prepare_manifests(data, vehicle_runs)
    return manifests

In [9]:
def offline_solver(payload):
    """
    Will solve with ORTOOLS.

    Args:
        payload: (JSON) see prepare_data_for_offline_solver()

    Returns:
        (JSON) resulting manifests for this date

    """
    data = create_data_model(pd.DataFrame.from_records(payload['requests']), pd.DataFrame.from_records(payload['driver_runs']), payload['date'], payload['depot'])
    df_manifests = run_vrp(data)
    df_manifests = df_manifests[df_manifests['booking_id'] != -1]
    df_requests = pd.DataFrame.from_records(payload['requests'])
    df_manifests = df_manifests.merge(df_requests, how='left', on='booking_id')
    df_manifests['time_window_start'] = df_manifests.apply(lambda row: row['pickup_time_window_start'] if (row['action'] == 'pickup') else row['dropoff_time_window_start'], axis=1)
    df_manifests['time_window_end'] = df_manifests.apply(lambda row: row['pickup_time_window_end'] if (row['action'] == 'pickup') else row['dropoff_time_window_end'], axis=1)
    df_manifests['node_id'] = df_manifests.apply(lambda row: row['pickup_node_id'] if (row['action'] == 'pickup') else row['dropoff_node_id'], axis=1)
    dfs = []
    for run_id in df_manifests['run_id'].unique():
        df_manifest = df_manifests[(df_manifests['run_id']==run_id)].sort_values(by='order', ascending=True)
        current_location = df_manifest.iloc[0]
        current_scheduled_time_tw = current_location['time_window_start']
        current_scheduled_time_d = list(filter(lambda x: x['run_id']==run_id, payload['driver_runs']))[0]['start_time'] + payload['time_matrix'][payload['depot']['node_id']][current_location['node_id']] + config.DWELL_TIME
        current_scheduled_time = max(current_scheduled_time_d, current_scheduled_time_tw)
        scheduled_times = [current_scheduled_time]
        for i in range(1, len(df_manifest)):
            current_location = df_manifest.iloc[i-1]
            next_location = df_manifest.iloc[i]
            current_scheduled_time = current_scheduled_time + payload['time_matrix'][current_location['node_id']][next_location['node_id']] + config.DWELL_TIME
            if current_scheduled_time < next_location['time_window_start']:
                current_scheduled_time = next_location['time_window_start']
            scheduled_times.append(current_scheduled_time)
        df_manifest['scheduled_time'] = scheduled_times
        dfs.append(df_manifest)
    df = pd.concat(dfs, ignore_index=True)
    return df[['run_id', 'booking_id', 'action', 'scheduled_time', 'node_id']].to_dict('records')

### Run Offline Solver

In [10]:
manifests = offline_solver(payload)

VRP: Running VRP to return a Timetable
VRP: Time Limit is 300
VRP: time limit will be 300
VRP: Create Routing Index Manager (pywrapcp.RoutingIndexManager) - Start
VRP: Created Routing Index Manager
VRP: Creating Routing Model (pywrapcp.RoutingModel) - Start
VRP: Created Routing Model
VRP: Adding Distance Dimension (routing.AddDimension) - Start
VRP: Added Distance Dimension
VRP: Adding Time Dimension (routing.AddDimension) - Start
VRP: Added Time Dimension
VRP: SetArcCostEvalatorOfAllVehicles
VRP: Defined Transit Callback
VRP: Adding Pickup/Delivery indexes for each request - Start
VRP: Finished with Pickup & Deliveries
VRP: Adding Time Windows - Start
VRP: Completed Time Windows
VRP: Completed Time Windows (Start and End Locations)
VRP: Instantiate Route Start and End Times - Start
VRP: Instantiated Route Start and End Times
VRP: am_demand_callback - Starting
VRP: Complete AM Capacity Constraints
VRP: Starting wc_demand_callback
VRP: Ended wc_demand_callback
VRP: Completed Capacity Co

In [11]:
manifests

[{'run_id': 0,
  'booking_id': 1129912,
  'action': 'pickup',
  'scheduled_time': 19800,
  'node_id': 5},
 {'run_id': 0,
  'booking_id': 1129871,
  'action': 'pickup',
  'scheduled_time': 20419,
  'node_id': 1},
 {'run_id': 0,
  'booking_id': 1129871,
  'action': 'dropoff',
  'scheduled_time': 20953,
  'node_id': 2},
 {'run_id': 0,
  'booking_id': 1129912,
  'action': 'dropoff',
  'scheduled_time': 21253,
  'node_id': 6},
 {'run_id': 0,
  'booking_id': 1126925,
  'action': 'pickup',
  'scheduled_time': 27000,
  'node_id': 23},
 {'run_id': 0,
  'booking_id': 1126925,
  'action': 'dropoff',
  'scheduled_time': 27639,
  'node_id': 24},
 {'run_id': 0,
  'booking_id': 1128512,
  'action': 'pickup',
  'scheduled_time': 28800,
  'node_id': 49},
 {'run_id': 0,
  'booking_id': 1130181,
  'action': 'pickup',
  'scheduled_time': 29464,
  'node_id': 37},
 {'run_id': 0,
  'booking_id': 1130181,
  'action': 'dropoff',
  'scheduled_time': 30361,
  'node_id': 38},
 {'run_id': 0,
  'booking_id': 112851

In [12]:
with open('./data/offline_manifests.pkl', 'wb') as f:
    pickle.dump(manifests,f)

## Metric Functions

In [13]:
def get_pt(pickup_lat, pickup_lon, dropoff_lat, dropoff_lon, action): 
    if action == 'pickup':
        r = {'lat': pickup_lat, 'lon': pickup_lon}
    elif action == 'dropoff':
        r = {'lat': dropoff_lat, 'lon': dropoff_lon}
    else:
        r = {'lat': config.DEPOT_LAT, 'lon': config.DEPOT_LON}
    return r
def manifest_report(df, payload):
    # df must have columns 'booking_id', 'scheduled_arrival_dt', 'action'
    
    # first sort by 'scheduled_time' so we don't have to assume df is in the correct order initially
    df = df.sort_values(by='scheduled_time')
    df['node_id'] = list(range(len(df)))
    
    # calculate 'occupancy' for manifest
    occupancy = [0]
    for i in range(1, len(df)):
        if df.iloc[i]['action'] == 'pickup':
            temp = occupancy[-1] + 1
        elif df.iloc[i]['action'] == 'dropoff':
            temp = occupancy[-1] - 1
        else:
            temp = 0
        occupancy.append(temp)
    df['occupancy'] = occupancy
    
    # need to get geolocation of each point so we can calculate distances, this requires merging with requests
    df_requests = pd.DataFrame(payload['requests'])
    df = pd.merge(df, df_requests, on='booking_id', how='left')
    df['pt'] = df.apply(lambda row: get_pt(row['pickup_pt']['lat'], row['pickup_pt']['lon'], row['dropoff_pt']['lat'], row['dropoff_pt']['lon'], row['action']), axis=1)
    df = df[['booking_id', 'scheduled_time', 'action', 'occupancy', 'node_id', 'pt']]
    
    # get travel_time_matrix and distance_matrix    
    distance_matrix = osrm.request_distance_matrix(df['pt'].tolist())

    # calculate number_of_passengers_served
    number_of_passengers_served = len(df[df['action']=='dropoff'])    
    total_distance, total_distance_without_passengers = 0, 0
    for i in range(len(df)-1):
        total_distance += distance_matrix[i][i+1]
        if df.iloc[i]['occupancy'] == 0:
            total_distance_without_passengers += distance_matrix[i][i+1]
    # Need to set up functions once osrm server is set up         
    from_depot = osrm.request_travel_distance({'lat': config.DEPOT_LAT, 'lon': config.DEPOT_LON}, df.iloc[0]['pt'])
    to_depot = osrm.request_travel_distance(df.iloc[-1]['pt'], {'lat': config.DEPOT_LAT, 'lon': config.DEPOT_LON})

    total_distance = total_distance + from_depot + to_depot
    total_distance_without_passengers = total_distance_without_passengers + from_depot + to_depot
    
    vehicle_miles_travelled = total_distance / 1609 # convert meters to miles
    vehicle_deadhead_miles = total_distance_without_passengers / 1609
    
    number_of_shared_rides = 0
    passenger_meters_travelled = 0
    for booking_id in df['booking_id'].unique():
        if booking_id != -1:
            pickup = df[(df['booking_id']==booking_id) & (df['action']=='pickup')].iloc[0]
            dropoff = df[(df['booking_id']==booking_id) & (df['action']=='dropoff')].iloc[0]
            dfrn = pd.DataFrame(payload['requests'])
            rn = dfrn[(dfrn['booking_id']==booking_id)].iloc[0]
            num_passengers = rn['am'] + rn['wc']
            passenger_meters_travelled += (distance_matrix[pickup['node_id']][dropoff['node_id']]) * num_passengers
            if (pickup['occupancy'] > 1) or (dropoff['occupancy'] > 0) or (dropoff['node_id'] - pickup['node_id'] != 1):
                number_of_shared_rides += 1
    
    passenger_miles_travelled = passenger_meters_travelled / 1609
    return {
        'VMT': vehicle_miles_travelled,
        'PMT': passenger_miles_travelled,
        'VMT_PMT_Ratio': vehicle_miles_travelled / passenger_miles_travelled,
        'VDM': vehicle_deadhead_miles,
        'number_of_passengers_served': number_of_passengers_served,
        'number_of_shared_rides': number_of_shared_rides,
        'Shared_Rate': int((number_of_shared_rides / number_of_passengers_served) * 100)
    }
    
def timetable_report(df, payload):
    # df must have columns 'booking_id', 'scheduled_arrival_dt', 'action', 'run_id'
    driver_runs_available = list({d['run_id'] for d in payload['driver_runs']})
    dfr = pd.DataFrame(payload['requests'])
    # dfr = dfr[dfr['leave_open']==False]
    vehicle_reports = []
    for run_id in driver_runs_available:
        df_run = df[df['run_id']==run_id]
        if len(df_run[df_run['action'].isin(['pickup', 'dropoff'])]) > 0:
            vehicle_report = manifest_report(
                df_run,
                payload,
            )
            vehicle_report['run_id'] = run_id
        else:
            vehicle_report = {
                'VMT': 0,
                'PMT': 0,
                'VMT_PMT_Ratio': 0,
                'VDM': 0,
                'number_of_passengers_served': 0,
                'number_of_shared_rides': 0,
                'Shared_Rate': 0,
                'run_id': run_id
            }
        vehicle_reports.append(vehicle_report)
            
    # calculate the metrics for the whole timetable
    df_metrics = pd.DataFrame(vehicle_reports)
    VMT = df_metrics['VMT'].sum()
    PMT = df_metrics['PMT'].sum()
    VDM = df_metrics['VDM'].sum()
    number_of_passengers_served = df_metrics['number_of_passengers_served'].sum()
    number_of_shared_rides = df_metrics['number_of_shared_rides'].sum()
    total_number_of_requests = len(dfr)
    timetable_report = {
        'VMT': VMT,
        'PMT': PMT,
        'VMT_PMT_Ratio': VMT / PMT,
        'VDM': VDM,
        'number_of_passengers_served': number_of_passengers_served,
        'total_number_of_requests': total_number_of_requests,
        'number_of_shared_rides': number_of_shared_rides,
        'Shared_Rate': int((number_of_shared_rides / number_of_passengers_served) * 100),
        'Service_Rate': int((number_of_passengers_served / total_number_of_requests) * 100)
    }
    
    result = {
        'timetable_report': timetable_report,
        'vehicle_reports': vehicle_reports
    }
    return result

### Get Metrics

In [14]:
df = pd.DataFrame(manifests)
result = timetable_report(df,payload)
result

{'timetable_report': {'VMT': 1306.161591050342,
  'PMT': 951.1640770665009,
  'VMT_PMT_Ratio': 1.37322426544818,
  'VDM': 756.994406463642,
  'number_of_passengers_served': 169,
  'total_number_of_requests': 169,
  'number_of_shared_rides': 97,
  'Shared_Rate': 57,
  'Service_Rate': 100},
 'vehicle_reports': [{'VMT': 113.69732753262896,
   'PMT': 84.40459912989435,
   'VMT_PMT_Ratio': 1.3470513301965288,
   'VDM': 65.68489745183344,
   'number_of_passengers_served': 17,
   'number_of_shared_rides': 12,
   'Shared_Rate': 70,
   'run_id': 0},
  {'VMT': 62.778123057799874,
   'PMT': 46.300807955251706,
   'VMT_PMT_Ratio': 1.3558753255120943,
   'VDM': 36.59850839030454,
   'number_of_passengers_served': 10,
   'number_of_shared_rides': 8,
   'Shared_Rate': 80,
   'run_id': 1},
  {'VMT': 79.64822871348663,
   'PMT': 67.05096333126166,
   'VMT_PMT_Ratio': 1.187875979051768,
   'VDM': 41.56619018023617,
   'number_of_passengers_served': 10,
   'number_of_shared_rides': 6,
   'Shared_Rate': 6